<a href="https://colab.research.google.com/github/MapleMoonSky/test_colaboratory_google/blob/master/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@title Fer2013 
'''
训练集：fer28709
验证集：3588
测试集：3588

1-1:
Test loss: 3.510471635837619
Test accuracy: 0.600613154960981
1-2:
Test loss: 3.490228995298728
Test accuracy: 0.5983835005574136
1-3:
Test loss: 3.5108614084845535
Test accuracy: 0.6064659977703456

tv：
1-3
Test loss: 3.484443254845701
Test accuracy: 0.6130471145887377
1-4:
Test loss: 3.4791799931223175
Test accuracy: 0.6201226309921962
1-5:
Test loss: 3.428256825858534
Test accuracy: 0.6089743589743589

2019/08/21/001
Test loss: 3.3394725171219943
Test accuracy: 0.6011705685618729

2019/08/21/002
Test loss: 3.4780489210301018
Test accuracy: 0.6117614269788183

'''

from __future__ import print_function
import string,os,sys
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
import random
import keras
import tensorflow as tf

config = tf.ConfigProto(allow_soft_placement=True)
#最多占gpu资源的70%
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)
#开始不会给tensorflow全部gpu资源 而是按需增加
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.models import load_model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D,AveragePooling2D
from keras.optimizers import Adam
from keras import backend as K
from keras.utils import plot_model
from keras.callbacks import TensorBoard
#K.set_image_dim_ordering('tf')
# the data, shuffled and split between train and test setstest


file_path='/content/drive/My Drive/test/fer2013.csv'
data=pd.read_csv(file_path,dtype='a')
label=np.array(data['emotion'])
img_data=np.array(data['pixels'])
N_sample = label.size
Face_data = np.zeros((N_sample, 48*48))
Face_label = np.zeros((N_sample,7), dtype=int)#7列

for i in range(N_sample):
 x = img_data[i]
 x = np.fromstring(x, dtype=float, sep=' ')
 x_max = x.max()
 x = x/(x_max+0.0001)#归一化
 Face_data[i] = x
 #重新编码，将类别向量（从0到nb_classes的整数向量）映射为二值类别矩阵
 z=int(label[i])
 Face_label[i,z]=1###二值化
 #img_x =np.reshape(x,(48, 48))
 #plt.subplot(5,5,i+1);
 #plt.imshow(img_x,cmap='gray')
#plt.show()

train_num=28709
valid_num=32298
test_num=35887
#train_num = 28710
#test_num = 3590




train_x = Face_data [0:valid_num, :]#分割训练集数据
train_y = Face_label [0:valid_num, :]#分割训练集label
valid_x = Face_data [train_num+1:valid_num, :]
valid_y = Face_label [train_num+1:valid_num, :]

test_x = Face_data[valid_num+1:test_num+1, :]#分割测试集数据
test_y = Face_label[valid_num+1:test_num+1, :]#分割测试集label


# input image dimensions
img_rows, img_cols = 48, 48



if K.image_data_format() == 'channels_first':
    x_train = train_x.reshape(train_x.shape[0], 1, img_rows, img_cols)
    x_test = test_x.reshape(test_x.shape[0], 1, img_rows, img_cols)
    x_valid=valid_x.reshape(valid_x.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = train_x.reshape(train_x.shape[0], img_rows, img_cols, 1)
    x_test = test_x.reshape(test_x.shape[0], img_rows, img_cols, 1)
    x_valid = valid_x.reshape(valid_x.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_valid= x_valid.astype('float32')


#x_train /= 255
#x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0],'valid samples')
print(x_test.shape[0], 'test samples')
y_train=train_y
y_test=test_y
y_valid=valid_y

# 将类别向量（从0到nb_classes的整数向量）映射为二值类别矩阵
#用编码器ong-hot重新编码如  9 编码为000000001
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

batch_size = 128
num_classes = 7
epochs = 200
model = Sequential()

#"convolution2d_10
model.add(Conv2D(64, kernel_size=(5, 5),
                 padding='same',
                 activation='relu',
                 input_shape=input_shape,
                 data_format='channels_last'))

model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),data_format='channels_last',padding='valid'))


#"convolution2d_11
model.add(Conv2D(64, kernel_size=(5, 5), padding='same',activation='relu',data_format='channels_last'))

model.add(AveragePooling2D(pool_size=(3, 3),strides=(2,2),data_format='channels_last',padding='valid'))

#"convolution2d_12
model.add(Conv2D(128, kernel_size=(5, 5), padding='same',activation='relu',data_format='channels_last'))

#maxpooling2d_4
model.add(AveragePooling2D(pool_size=(3, 3),strides=(2,2),data_format='channels_last',padding='valid'))

#convolution2d_13


model.add(Flatten())

#dense_4(设置输出层的维度)
model.add(Dense(128, activation='relu'))

#dropout_3
model.add(Dropout(0.5))

#dense_5稠密层


#dense_6
model.add(Dense(7, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0010000000474974513,beta_1=0.8999999761581421,beta_2=0.9990000128746033,epsilon=1e-08),
              metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


json_string=model.to_json()
open('/content/drive/My Drive/test/model1/model1-5.json','w').write(json_string)
model.save_weights('/content/drive/My Drive/test/model1/model1-5.h5')
plot_model(model,to_file='/content/drive/My Drive/test/model1/model.png',show_shapes=True)

x_train shape: (32298, 48, 48, 1)
32298 train samples
3588 valid samples
3588 test samples
Train on 32298 samples, validate on 3588 samples
Epoch 1/200
32298/32298 [==============================] - 10s 323us/step - loss: 1.8044 - acc: 0.2487 - val_loss: 1.6983 - val_acc: 0.3258
Epoch 2/200
32298/32298 [==============================] - 10s 298us/step - loss: 1.6521 - acc: 0.3492 - val_loss: 1.5292 - val_acc: 0.4055
Epoch 3/200
32298/32298 [==============================] - 10s 298us/step - loss: 1.5356 - acc: 0.4079 - val_loss: 1.4347 - val_acc: 0.4457
Epoch 4/200
32298/32298 [==============================] - 10s 297us/step - loss: 1.4389 - acc: 0.4485 - val_loss: 1.3833 - val_acc: 0.4574
Epoch 5/200
32298/32298 [==============================] - 10s 296us/step - loss: 1.3710 - acc: 0.4773 - val_loss: 1.2990 - val_acc: 0.4972
Epoch 6/200
32298/32298 [==============================] - 10s 297us/step - loss: 1.3206 - acc: 0.4976 - val_loss: 1.2426 - val_acc: 0.5229
Epoch 7/200
32298/32

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13179504866963555174, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9197910189893618559
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10350872910569264613
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11326753997
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14506816406449921320
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Fer2013首次测试

日期：2019.8.21

----

### Python 3 Google Compute Engine 后端(GPU)

内存：12.72G

每个epoch时间9-10s

运行完大约需要34分钟

实际运行完成时间1928.73s(32.13min)

---

### YCKJ2033 
CPU: Intel(R) Core(TM) i5-8500 CPU.00GHz 3.00 GHz

GPU：无

内存：16G

每个epoch时间188-192s

运行完大约需要10.4小时